## Imports

In [6]:
!echo $CUDA_VISIBLE_DEVICES

2,3


In [7]:
import torch
from torch import nn

In [8]:
%run -n train_report_generation.py

In [9]:
DEVICE = torch.device('cuda')
DEVICE

device(type='cuda')

## Load stuff

### Load data

In [17]:
%run ./datasets/iu_xray.py

In [31]:
dataset_kwargs = {
    'max_samples': 10,
    'frontal_only': False,
}

train_dataset = IUXRayDataset(dataset_type='train', **dataset_kwargs)
val_dataset = IUXRayDataset(dataset_type='val', vocab=train_dataset.get_vocab(),
                            **dataset_kwargs)
train_dataset.size(), val_dataset.size()

((20, 10), (18, 10))

In [32]:
BS = 20

train_dataloader = create_pad_dataloader(train_dataset, batch_size=BS)
val_dataloader = create_pad_dataloader(val_dataset, batch_size=BS)
train_dataloader.dataset.size()

(20, 10)

### Load model

In [22]:
%run ./models/classification/__init__.py
%run ./models/report_generation/decoder_lstm.py
%run ./models/report_generation/decoder_lstm_att.py
%run ./models/report_generation/cnn_to_seq.py
%run ./models/checkpoint/__init__.py

#### Load CNN

In [10]:
cnn_run_name = '0706_134245_covid-kaggle_tfs-small_lr1e-06'
debug_run = True

compiled_cnn = load_compiled_model_classification(cnn_run_name,
                                                  debug=debug_run,
                                                  device=DEVICE)
cnn = compiled_cnn.model

#### Or create CNN

In [23]:
cnn = init_empty_model('densenet-121', # resnet-50
                       labels=[],
                       imagenet=True,
                       freeze=False,
                       ).to(DEVICE)

#### Create Decoder

In [11]:
decoder = LSTMDecoder(len(train_dataset.word_to_idx), 100, 100, cnn.features_size,
                      teacher_forcing=True).to(DEVICE)

In [24]:
decoder_att = LSTMAttDecoder(len(train_dataset.word_to_idx), 100, 100, cnn.features_size,
                             teacher_forcing=True).to(DEVICE)

#### Full model

In [25]:
model = CNN2Seq(cnn, decoder_att).to(DEVICE)

In [26]:
model = nn.DataParallel(model)

In [27]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)

compiled_model = CompiledModel(model, optimizer)

## Train

In [33]:
%%time

train_metrics, val_metrics = train_model('debugging',
                                         compiled_model,
                                         train_dataloader,
                                         val_dataloader,
                                         n_epochs=1,
                                         dryrun=True,
                                         save_model=False,
                                         debug=True,
                                         device=DEVICE)

--------------------------------------------------
Training...
Finished epoch 1/1 loss 6.6042 6.5588, bleu 0.0200 0.0273, 0h 0m 1s
Average time per epoch:  0h 0m 1s
--------------------------------------------------
CPU times: user 11.6 s, sys: 11.3 s, total: 22.9 s
Wall time: 1.55 s


In [34]:
val_metrics

{'loss': 6.558760166168213,
 'word_acc': 0.15086206896551724,
 'bleu1': 0.1093749999996582,
 'bleu2': 5.939887041374782e-10,
 'bleu3': 1.0555489364351294e-12,
 'bleu4': 4.487549787118121e-14,
 'bleu': 0.027343750148686834,
 'rougeL': 0.12996192322854097,
 'ciderD': 0.0}

In [35]:
train_metrics

{'loss': 6.604184627532959,
 'word_acc': 0.1509433962264151,
 'bleu1': 0.07999999999980001,
 'bleu2': 4.5291081365669157e-10,
 'bleu3': 8.142303025215148e-13,
 'bleu4': 3.4754417571031264e-14,
 'bleu': 0.02000000011338995,
 'rougeL': 0.10896939630225835,
 'ciderD': 0.0}

## Test samples

In [19]:
idx_to_word = {v: k for k, v in train_dataset.get_vocab().items()}
# idx_to_word

In [20]:
def idx_to_text(idxs):
    return ' '.join([idx_to_word[int(g.item())] for g in idxs])

In [85]:
idx = 150

In [86]:
image, report = train_dataset[idx]
image.size(), report.size()

(torch.Size([3, 512, 512]), torch.Size([13]))

In [87]:
images = image.unsqueeze(0).to(DEVICE)
generated, scores = model(images, report.size()[0])
_, generated = generated.max(dim=2)
generated = generated.squeeze(0).cpu()
# print(generated.size())
# print(generated)

idx_to_text(generated)

'both lungs are clear and expanded . heart and mediastinum normal . END'

In [88]:
idx_to_text(report)

'both lungs are clear and expanded . heart and mediastinum normal . END'

### Search reports with a certain pattern

In [24]:
from tqdm.notebook import tqdm
import re

In [65]:
# target = re.compile(r'\A[a-zA-Z]+ size is normal')
target = re.compile('both lungs are clear and expanded')
found = []

for report in train_dataset.reports:
    report = idx_to_text(report['tokens_idxs'])
    if target.search(report):
        found.append(report)

len(found)

162

In [70]:
found_diff = list(set(found))
len(found_diff)

19

In [76]:
found_diff[5]

'chest . both lungs are clear and expanded with no pleural air collections or parenchymal consolidations . heart and mediastinum remain normal . lumbosacral spine . xxxx , disc spaces , and alignment are normal . sacrum and sacroiliac joints are normal . END'

## Debug metrics

In [8]:
from ignite.metrics import MetricsLambda

In [6]:
%run metrics/report_generation/bleu.py

In [9]:
bleu_up_to_4 = Bleu(n=4)

In [10]:
bleu1 = MetricsLambda(lambda x: x[0], bleu_up_to_4)
bleu2 = MetricsLambda(lambda x: x[1], bleu_up_to_4)
bleu3 = MetricsLambda(lambda x: x[2], bleu_up_to_4)
bleu4 = MetricsLambda(lambda x: x[3], bleu_up_to_4)
bleuAvg = MetricsLambda(lambda x: torch.mean(x), bleu_up_to_4)

## Debug attention

In [15]:
from torch import nn

In [146]:
%run models/report_generation/decoder_lstm_att.py

In [147]:
decoder = LSTMAttDecoder(200, 100, 100, (2048, 16, 16))

In [148]:
images = torch.randn(3, 2048, 16, 16).float()
images.size()

torch.Size([3, 2048, 16, 16])

In [149]:
out, scores = decoder(images, 10)
out.size(), scores.size()

(torch.Size([3, 10, 200]), torch.Size([3, 10, 16, 16]))

In [140]:
feats, scores = att(images, h_state)
feats.size(), scores.size()

(torch.Size([3, 2048]), torch.Size([3, 16, 16]))